# kd Tree build
Input: dataset: $ T = {x_1, x_2, ..., x_N} $,  and $x_i = (x_i^{(1)}, x_i^{(2)}, ..., x_i^{(k)})^T$, and $i = 1,2, ..., N$
Output: kd tree<br/>
Alogrithm:<br/>
(1) 开始： 构造根节点， 根节点对应于包含T的k维空间的超矩形区域。<br/>
选择$x^{(1)}$ 为坐标轴， 以T中所有实例的$x^{(1)}$坐标的中位数为切分点， 将根节点对应的超矩形区域切分为两个子区域，切分由通过切分点并与坐标轴$x^{(1)}$垂直的超平面实现。<br/>
由根节点生成深度为1的左、右子节点： <br/>
左子节点对应坐标$x^{(1)}$小于切分点的子区域， <br/>
右子节点对应坐标$x^{(1)}$大于切分点的子区域， <br/>
将落在切分超平面上的实例点保存在根节点。 <br/>
（2） 重复：  对深度为j的节点， 选择$x^{(l)}$为切分的坐标轴，  $l = j(mod k) + 1$, 以该节点的区域中所有实例的$x^{(l)}$坐标中位数为切分点， 将根节点对应的超矩形区域切分为两个子区域，切分由通过切分点并与坐标轴$x^{(l)}$垂直的超平面实现。 <br/>
由该节点生成深度为$j + 1$的左、右子节点： <br/>
左子节点对应坐标$x^{(l)}$小于切分点的子区域， <br/>
右子节点对应坐标$x^{(l)}$大于切分点的子区域， <br/>
将落在切分超平面上的实例点保存在根节点。 <br/>
(3) 直到两个子区域没有实例存在时停止。 从而形成kd树的区域划分。
# kd Tree search
Alogrithm:<br/>

Input: kd tree<br/>
Output: nearest neighbors of x<br/>
(1)在kd树中找出包含目标点x的叶结点:从根结点出发，递归地向下访问kd树.若目标点x当前维的坐标小于切分点的坐标，则移动到左子结点，否则移动到右子结点，直到子结点为叶结点为止.<br/>
(2)以此叶结点为“当前最近点”.<br/>
(3)递归地向上回退，在每个结点进行以下操作:<br/>
    (a)如果该结点保存的实例点比当前最近点距离目标点更近，则以该实例点为“当前最近点”.<br/>
    (b)当前最近点一定存在于该结点一个子结点对应的区域.检查该子结点的父结点的另一子结点对应的区域是否有更近的点，具体地，检查另一子结点对应的区域是否与以目标点为球心、以目标点与“当前最近点”间的距离为半径的超球体相交.<br/>
       如果相交，可能在另一个子结点对应的区域内存在距目标点更近的点，移动到另一个子结点，接着，递归地进行最近邻搜索;<br/>
       如果不相交，向上回退。<br/>
(4)当回退到根结点时，搜索结束，最后的“当前最近点”即为x的最近邻点.<br/>

In [10]:
import numpy as np
import pprint

In [8]:
# build kd tree
class kdTree:
    """
    k : k dimensions
    method: alternate(坐标轴轮替)/variance（最大方差轴）
    """
    def __init__(self, k = 2,  method = "alternate"):
        self.k = k
        self.method = method

    def build(self, points, depth = 0):
        n = len(points)
        if n <= 0:
            return None

        if self.method == "alternate":
            axis = depth % self.k
        elif self.method == "variance":
            axis = np.argmax(np.var(points, axis=0), axis=0)

        sorted_points = sorted(points, key = lambda point: point[axis])
        return {
            "point": sorted_points[n // 2],
            "left": self.build(sorted_points[:n//2], depth + 1),
            "right": self.build(sorted_points[n//2 + 1:], depth + 1)
        }

In [13]:
# test example
data = np.array([[2,3],[5,4],[9,6],[4,7],[8,1],[7,2]])

kd1 = kdTree(k = 2, method= "alternate")
tree1 = kd1.build(data)

kd2 = kdTree(k = 2, method = "variance")
tree2 = kd2.build(data)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(tree1)
pp.pprint(tree2)

{   'left': {   'left': {'left': None, 'point': array([2, 3]), 'right': None},
                'point': array([5, 4]),
                'right': {'left': None, 'point': array([4, 7]), 'right': None}},
    'point': array([7, 2]),
    'right': {   'left': {'left': None, 'point': array([8, 1]), 'right': None},
                 'point': array([9, 6]),
                 'right': None}}
{   'left': {   'left': {'left': None, 'point': array([2, 3]), 'right': None},
                'point': array([5, 4]),
                'right': {'left': None, 'point': array([4, 7]), 'right': None}},
    'point': array([7, 2]),
    'right': {   'left': {'left': None, 'point': array([8, 1]), 'right': None},
                 'point': array([9, 6]),
                 'right': None}}


In [14]:
# define distance metrix
def distance(x, y, p=2):
    try:
        dis = np.power(np.sum(np.power(np.abs((x - y)), p), 1), 1/p)
    except:
        dis = np.power(np.sum(np.power(np.abs((x - y)), p)), 1/p)

    return dis

In [18]:
# kd tree search
class searchkdTree:
    """
    search closest point
    """
    def __init__(self, k = 2):
        self.k = k

    def __closer_distance(self, pivot, p1, p2):
        if p1 is None:
            return p2
        if p2 is None:
            return p1

        d1 = distance(pivot, p1)
        d2 = distance(pivot, p2)

        if d1 < d2:
            return p1
        else:
            return p2

    def fit(self, root, point, depth=0):
        if root is None:
            return None

        axis = depth % self.k

        next_branch = None
        opposite_branch = None

        if point[axis] < root["point"][axis]:
            next_branch = root["left"]
            opposite_branch = root["right"]
        else:
            next_branch = root["right"]
            opposite_branch = root["left"]

        best = self.__closer_distance(point,
                                      self.fit(next_branch, 
                                               point,
                                               depth + 1),
                                      root["point"])
        if distance(point, best) > abs(point[axis] - root["point"][axis]):
            best = self.__closer_distance(point,
                                          self.fit(opposite_branch, 
                                               point,
                                               depth + 1),
                                      best)
        return best

In [19]:
# test
point = [3., 4.5]
search = searchkdTree()
best = search.fit(tree1, point, depth=0)
print(best)

[2 3]
